In [1]:
import sys

import metal
import os
# Import other dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
os.environ['METALHOME'] = '/dfs/scratch1/saelig/slicing/metal/'
# Set random seed for notebook
SEED = 123

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Load Data


In [42]:
from skimage import io, transform
import torchvision.transforms as transforms
import numpy as np

opj = os.path.join
HOME_DIR = '/dfs/scratch1/saelig/slicing/'
DATASET_DIR = opj(HOME_DIR,'CUB_200_2011')
IMAGES_DIR = opj(DATASET_DIR, 'images')
TENSORS_DIR = opj(HOME_DIR, 'birds_data')
MODELS_DIR = opj(HOME_DIR, 'birds_models')

#Size of eac
#image_list = np.loadtxt(os.path.join(DATASET_DIR, 'images.txt'), dtype=str)
#train_test_split = np.loadtxt(os.path.join(DATASET_DIR, 'train_test_split.txt'), dtype=int)
#labels = np.loadtxt(os.path.join(DATASET_DIR, 'image_class_labels.txt'), dtype=int)


train_image_ids = torch.load(opj(TENSORS_DIR,'train_image_ids.pt'))
valid_image_ids = torch.load(opj(TENSORS_DIR,'valid_image_ids.pt'))
test_image_ids = torch.load(opj(TENSORS_DIR,'test_image_ids.pt'))
X_train = torch.load(opj(TENSORS_DIR,'X_train.pt'))
X_valid = torch.load(opj(TENSORS_DIR,'X_valid.pt'))
X_test = torch.load(opj(TENSORS_DIR,'X_test.pt'))
Y_train = torch.load(opj(TENSORS_DIR,'Y_train.pt'))
Y_valid = torch.load(opj(TENSORS_DIR,'Y_valid.pt'))
Y_test = torch.load(opj(TENSORS_DIR,'Y_test.pt'))



Let's create the payloads. First we need to put the attribute information into an easy to deal with data structure.

In [54]:
attrs_array = np.loadtxt(os.path.join(DATASET_DIR, 'attributes/image_attribute_labels.txt'), usecols=(0,1,2), dtype=int)

Let's create a dictionary to make it easier to figure out which samples have which attributes.

In [24]:
NUM_ATTRIBUTES = 312

#format: <image_id>,  <attribute_id>,  <is_present>

attrs_dict = {} #dict mapping attribute id to a set of image_ids that have that attribute

# for attr in range(1, NUM_ATTRIBUTES + 1):
#     temp = attrs_array[(attrs_array[:, 1] == attr) & (attrs_array[:,2] == 1)]
#     print(temp)
#     break

for (image_id, attr_id, is_present) in attrs_array:
    if is_present == 1:
        if attr_id in attrs_dict:
            attrs_dict[attr_id].add(image_id)
        else:
            attrs_dict[attr_id] = {image_id}

Create payload abstraction for slices based on the binary attributes.

In [53]:
from metal.mmtl.payload import Payload
from metal.mmtl.data import MmtlDataLoader, MmtlDataset
from pprint import pprint

payloads = []
splits = ["train", "valid", "test"]
X_splits = X_train, X_valid, X_test
Y_splits = Y_train, Y_valid, Y_test

task_name = 'BirdClassificationTask'
labels_to_tasks = {"labelset_gold": task_name}

for i in range(3):
    payload_name = f"Payload{i}_{splits[i]}"
    task_name = task0.name
    X_dict = {'data': X_splits[i]}
    Y_dict = {'labelset_gold': Y_splits[i]}
    
    if splits[i] == 'train':
        image_ids = train_image_ids
    elif splits[i] == 'valid':
        image_ids = valid_image_ids
    else:
        image_ids = test_image_ids
    for attr_id in range(1, NUM_ATTRIBUTES + 1):
        f = lambda x: 1 if x in attrs_dict[attr_id] else 0
        mask = list(map(f, image_ids.tolist()))
        mask = torch.tensor(mask)
        slice_labelset_name = f"labelset:{attr_id}:pred"
        slice_task_name = f"{task_name}:{attr_id}:pred"
        Y_dict[slice_labelset_name] = mask * Y_splits[i]
        labels_to_tasks[slice_labelset_name] = slice_task_name
        
        mask[mask == 0] = 2 #to follow Metal convention
        slice_labelset_name = f"labelset:{attr_id}:ind"
        slice_task_name = f"{task_name}:{attr_id}:ind"
        Y_dict[slice_labelset_name] = mask 
        labels_to_tasks[slice_labelset_name] = slice_task_name
        
    
    dataset = MmtlDataset(X_dict, Y_dict)
    data_loader = MmtlDataLoader(dataset, batch_size=32)
    payload = Payload(payload_name, data_loader, labels_to_tasks, splits[i])
    payloads.append(payload)


Let's load our baseline model

In [81]:
model = torch.load(opj(MODELS_DIR,'resnet18_lr_1e-3_patience10.pt')) #achieves 17% accuracy on test set

In [82]:
model.score(payloads[1], metrics='BirdClassificationTask/Payload1_valid/labels/accuracy') #score on validation set

['BirdClassificationTask/Payload1_valid/labels/accuracy']
BirdClassificationTask/Payload1_valid/labels/accuracy


Exception: Could not find the specified label_set labels in payload Payload(Payload1_valid: labels_to_tasks=[{'labelset_gold': 'BirdClassificationTask', 'labelset:1:pred': 'BirdClassificationTask:1:pred', 'labelset:1:ind': 'BirdClassificationTask:1:ind', 'labelset:2:pred': 'BirdClassificationTask:2:pred', 'labelset:2:ind': 'BirdClassificationTask:2:ind', 'labelset:3:pred': 'BirdClassificationTask:3:pred', 'labelset:3:ind': 'BirdClassificationTask:3:ind', 'labelset:4:pred': 'BirdClassificationTask:4:pred', 'labelset:4:ind': 'BirdClassificationTask:4:ind', 'labelset:5:pred': 'BirdClassificationTask:5:pred', 'labelset:5:ind': 'BirdClassificationTask:5:ind', 'labelset:6:pred': 'BirdClassificationTask:6:pred', 'labelset:6:ind': 'BirdClassificationTask:6:ind', 'labelset:7:pred': 'BirdClassificationTask:7:pred', 'labelset:7:ind': 'BirdClassificationTask:7:ind', 'labelset:8:pred': 'BirdClassificationTask:8:pred', 'labelset:8:ind': 'BirdClassificationTask:8:ind', 'labelset:9:pred': 'BirdClassif